<a href="https://colab.research.google.com/github/aripenguin/Data602/blob/main/assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 7**  
# **Weeks 8 & 9 - Pandas**  

**Introduction**  
The data I am using is *Parks Inspection Program – Daily Immediate Attentions* from NYC Open Data (https://data.cityofnewyork.us/City-Government/Parks-Inspection-Program-Daily-Immediate-Attention/s6dm-mdan/about_data). This dataset from the NYC Department of Parks and Recreation contains immediante attention hazards such as hateful graffiti or _. I choose this dataset because I wanted to put something from the NYC Open Data as local data sounded interesting to me. I wanted to look up datasets by departments and choose to check from Department of the Parks and Recreation because I am a big fan of the series Parks and Recreation. This one sound interesting and after downloading it, I really wanted to clean it up.

**Data Exploration**  

In [2]:
import pandas as pd

url = 'https://raw.githubusercontent.com/aripenguin/Data602/main/Parks_Inspection_Program___Daily_Immediate_Attentions_20240323.csv'
parks_hazards = pd.read_csv(url)

print("There are {} columns in this Daily Immediate Attentions dataset.".format(parks_hazards.shape[1]))
print("The column names are: {}".format(list(parks_hazards)))
print("There are {} rows in this Daily Immediate Attentions dataset.".format(parks_hazards.shape[0]))

There are 13 columns in this Daily Immediate Attentions dataset.
The column names are: ['Type', 'Inspection ID', 'Type Key', 'LastFieldInspectionDate', 'DateTimeEntered', 'iaAddedDate', 'OtherComments', 'Priority', 'Feature', 'WorkOrderNumber', 'Extended', 'SignedOff', 'orgtbl']
There are 94133 rows in this Daily Immediate Attentions dataset.


<ipython-input-2-ac33202cb8c8>:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  parks_hazards = pd.read_csv(url)


Below is an example of what the rows in the Daily Immediate Attentions dataset looks like.

As you can see, there are five columns (LastFieldInspectionDate, DateTimeEntered, OtherComments, WorkOrderNumber, SignedOff) that have some missing values.

Something else to note that can be cleaned is the last two columns (SignedOff, orgtbl). They hold the same data, whether or not the issue is resolved or not. This means they can be combined later.

In [3]:
print(parks_hazards.head)

<bound method NDFrame.head of       Type  Inspection ID  Type Key LastFieldInspectionDate  \
0       IA         176318    157665                     NaN   
1       IA         180483    161535                     NaN   
2       IA         177264    158684                     NaN   
3       IA         176927    158258                     NaN   
4       IA         141753    123269                     NaN   
...    ...            ...       ...                     ...   
94128   IA         181250    162221     2023-12-20T00:00:00   
94129   IA         181357    162341     2023-12-21T00:00:00   
94130   IA         181439    162417     2023-12-21T00:00:00   
94131   IA         181457    162434     2024-01-11T00:00:00   
94132   IA         181568    162522     2024-01-03T00:00:00   

               DateTimeEntered              iaAddedDate  \
0                          NaN  2023-04-13 23:00:00.000   
1                          NaN  2023-10-26 23:00:00.000   
2                          NaN  2023

**Summary statistics**  
 There isn't a lot of columns that would be worth doing numerical summary statistics in this dataset.

 For the priority column, we have three options for data (Z, 1, 2).  
 First, I will change all the Z/zero priority to 0 and save all the data in this column to type integer so we can look at the mean, median, and quantiles.

 I will revisit summary statistics for a new column I wish to make later.  


In [4]:
parks_hazards.loc[parks_hazards['Priority'] == 'Z', 'Priority'] = 0
parks_hazards['Priority'] = pd.to_numeric(parks_hazards['Priority'], errors='coerce', downcast='integer')
#parks_hazards['Priority'] = parks_hazards['Priority'].astype(int)
print(parks_hazards.loc[101,:])

print("The mean of the Priority column: {}".format(parks_hazards['Priority'].mean()))
print("The median of the Priority column: {}".format(parks_hazards['Priority'].median()))
print("The quantiles of the Priority column: \n{}".format(parks_hazards['Priority'].quantile([.1, .25, .5, .75])))

Type                                      Graffiti
Inspection ID                                55038
Type Key                                    142577
LastFieldInspectionDate        2007-11-05T00:00:00
DateTimeEntered            2008-04-24 13:44:58.140
iaAddedDate                2022-05-03 23:00:00.000
OtherComments                                  NaN
Priority                                       0.0
Feature                                   Graffiti
WorkOrderNumber                                NaN
Extended                                     False
SignedOff                                     True
orgtbl                                    Resolved
Name: 101, dtype: object
The mean of the Priority column: 1.8791819980007232
The median of the Priority column: 2.0
The quantiles of the Priority column: 
0.10    2.0
0.25    2.0
0.50    2.0
0.75    2.0
Name: Priority, dtype: float64


**Data Wrangling**  

Instead of working with 94,133 rows, I will subset this dataset to 10,000 rows to make it easier to work with.



In [44]:
#subset to 10,000
parks_hazards_subset = parks_hazards.sample(n = 10000)

print("1. Modify multiple column names.")
#organized the names of the date columns to match and DateTimeEntered did not describe the column well (column is for last action to address the hazard)
#Inspection ID and Type Key are modified to match the other names (no spaces)
parks_hazards_subset.rename(columns = {'DateTimeEntered':'LastActionDate', 'iaAddedDate':'AddedDate', 'Inspection ID':'InspectionID', 'Type Key':'TypeKey'}, inplace = True)
print(list(parks_hazards_subset))

print("\n2. Look at the structure of your data – are any variables improperly coded? Such as strings or characters? Convert to correct structure if needed.")
print(parks_hazards_subset.dtypes)
#Priority is set as a string but should be numeric
parks_hazards_subset['Priority'] = pd.to_numeric(parks_hazards_subset['Priority'], errors='coerce', downcast='integer')
#SignedOff is set as a string but should be a bool with the empty rows being False
parks_hazards_subset.loc[parks_hazards_subset['SignedOff'] != None, 'SignedOff'] = True
parks_hazards_subset['SignedOff'].fillna(False, inplace = True)
#all three date columns are not set as datetime
import datetime
parks_hazards_subset['LastFieldInspectionDate'] = pd.to_datetime(parks_hazards_subset['LastFieldInspectionDate'])
parks_hazards_subset['LastActionDate'] = pd.to_datetime(parks_hazards_subset['LastActionDate'])
parks_hazards_subset['AddedDate'] = pd.to_datetime(parks_hazards_subset['AddedDate'])
print(parks_hazards_subset.dtypes)

print("\n3. Fix missing and invalid values in data.")
print("Before fixing null values:\n{}".format(parks_hazards_subset.isnull().sum()))
#We already fixed SignedOff above, but LastFieldInspectionDate, LastActionDate, and OtherComments need
parks_hazards_subset['LastFieldInspectionDate'].fillna(parks_hazards_subset['LastActionDate'], inplace = True)
parks_hazards_subset['OtherComments'].fillna('N/A', inplace = True)
parks_hazards_subset['AddedDate'].fillna(parks_hazards_subset['LastActionDate'], inplace = True)
parks_hazards_subset['Feature'].fillna('N/A', inplace = True)
parks_hazards_subset['LastActionDate'].fillna(parks_hazards_subset['AddedDate'], inplace = True)
print("After fixing null values expect WorkOrderNumber:\n{}".format(parks_hazards_subset.isnull().sum()))

print("\n4. Create new columns based on existing columns or calculations.")
import datetime
#Add a column that shows the time between AddedDate to LastFieldInspectionDate: TimeSinceLastFieldInspection
parks_hazards_subset['TimeSinceLastFieldInspection'] = parks_hazards_subset['LastFieldInspectionDate'] - parks_hazards_subset['AddedDate']
#Add a column that shows the time between AddedDate to LastActionDate: TimeSinceLastAction
parks_hazards_subset['TimeSinceLastAction'] = parks_hazards_subset['LastActionDate'] - parks_hazards_subset['AddedDate']
#print(parks_hazards_subset.head(1))
print(parks_hazards_subset.sort_values(by = 'TimeSinceLastAction', ascending = False).head())

print("\n5. Drop column(s) from your dataset.")
print("There are {} columns in this dataset.".format(parks_hazards_subset.shape[1]))
#orgtbl holds the same data as SignedOff & WorkOrderNumber as very few rows use it (they can add it to OtherComments if there is one)
parks_hazards_subset['OtherComments'] = parks_hazards_subset['OtherComments'] + ' ' + parks_hazards_subset['WorkOrderNumber']
parks_hazards_subset.drop(['orgtbl', 'WorkOrderNumber'], axis=1, inplace=True)
print("There are {} columns after getting rid of orgtbl.".format(parks_hazards_subset.shape[1]))
print(list(parks_hazards_subset))

print("\n6. Drop a row(s) from your dataset.")
print("There are {} rows in this dataset.".format(parks_hazards_subset.shape[0]))
parks_hazards_subset = parks_hazards_subset.drop(parks_hazards_subset.index[[1,2,3]])
print("There are {} rows after the drop command.".format(parks_hazards_subset.shape[0]))

print("\n7. Sort your data based on multiple variables.")
print(parks_hazards_subset.sort_values(by = ['AddedDate', 'LastActionDate'], ascending = [True, False]).head(10))

print("\n8. Filter your data based on some condition.")
#I want to filter and save all non-SignedOff hazard rows
nonSignedOff = parks_hazards_subset.loc[parks_hazards_subset['SignedOff'] == False]
print("There is {} non-SignedOff hazards in this dataset.".format(nonSignedOff.shape[0]))
print(nonSignedOff.head())

print("\n9. Convert all the string values to upper or lower cases in one column.")
parks_hazards_subset['Type'] = parks_hazards_subset['Type'].str.lower()
print(parks_hazards_subset['Type'].head(5))

print("10. Check whether numeric values are present in a given column of your dataframe.")
priority_notnull = pd.to_numeric(parks_hazards_subset['Priority'], errors='coerce').notnull().all()
print("Are there numeric values in parks_hazards_subset['Priority']? {}".format(priority_notnull))

print("11. Group your dataset by one column, and get the mean, min, and max values by group: Groupby() & agg() or .apply()")
parks_hazards_types = parks_hazards_subset.groupby('Type')
print("The groups formed from grouping by Type are:\n{}".format(parks_hazards_types['Type'].unique()))
#get mean, min, and max values for Priority
types_priority = parks_hazards_types.agg({'Priority': ['mean','min', 'max']})
print("The mean, min, and max values of Priority by Type groups:\n{}".format(types_priority))
#get mean, min, and max values for TimeSinceLastFieldInspection
types_timeSinceLastFieldInspection = parks_hazards_types.agg({'TimeSinceLastFieldInspection': ['mean','min', 'max']})
print("The mean, min, and max values of TimeSinceLastFieldInspection by Type groups:\n{}".format(types_timeSinceLastFieldInspection))
#get mean, min, and max values for TimeSinceLastAction
types_timeSinceLastAction = parks_hazards_types.agg({'TimeSinceLastAction': ['mean','min', 'max']})
print("The mean, min, and max values of TimeSinceLastAction by Type groups:\n{}".format(types_timeSinceLastAction))


print("12. Group your dataset by two columns and then sort the aggregated results within the groups.")
parks_hazards_types_feature = parks_hazards_subset.groupby(['Type','Feature'], sort=True)
print("The groups formed from grouping by Type then Feature are:\n{}".format(parks_hazards_types_feature['Type','Feature'].size().reset_index().sort_values(['Type','Feature'], ascending = [True,False])))




1. Modify multiple column names.
['Type', 'InspectionID', 'TypeKey', 'LastFieldInspectionDate', 'LastActionDate', 'AddedDate', 'OtherComments', 'Priority', 'Feature', 'WorkOrderNumber', 'Extended', 'SignedOff', 'orgtbl']

2. Look at the structure of your data – are any variables improperly coded? Such as strings or characters? Convert to correct structure if needed.
Type                        object
InspectionID                 int64
TypeKey                      int64
LastFieldInspectionDate     object
LastActionDate              object
AddedDate                   object
OtherComments               object
Priority                   float64
Feature                     object
WorkOrderNumber             object
Extended                      bool
SignedOff                   object
orgtbl                      object
dtype: object
Type                               object
InspectionID                        int64
TypeKey                             int64
LastFieldInspectionDate    datetime6

<ipython-input-44-933a10510baf>:89: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  print("The groups formed from grouping by Type then Feature are:\n{}".format(parks_hazards_types_feature['Type','Feature'].size().reset_index().sort_values(['Type','Feature'], ascending = [True,False])))


**Conclusions**  

After exploring your dataset, I now have a clearer understanding of immediante action hazards. They can typically smaller non life-threatening hazards that can be, not always, deal with in a short timeframe. The most interesting part to be was the mean, min, and max for the new time difference columns I created. I was able to see which hazard types and features get dealt with the quickest compared to others. If I had more time, I would have liked to further invest that by doing more summary statistics and visualized data. Also, since this dataset has old and very recent records, I would want to look more at response timeframes across different years. Have it gotten easier or harder to fix different hazards?